In [1]:
import xarray as xr
import numpy as np
import os
from metpy import calc as mpcalc

In [2]:
if os.path.basename(os.path.expanduser("~")) == 'bfildier':
    inputdir = '/Users/bfildier/Data/EUREC4A/merged/sondes'
    radiosonde_path = os.path.join(inputdir,'all_radiosondes.nc')
    dropsonde_path = os.path.join(inputdir,'all_dropsondes.nc')
else:    
    inputdir = "/run/media/ludo/DATA/google-drive/Thèse/EUREC4a/github/Input/"
    dropsonde_path = os.path.join(inputdir, "Dropsondes", "all_dropsondes.nc")
    radiosonde_path = os.path.join(inputdir, "Radiosondes", "all_radiosondes.nc")


In [4]:
def prepare_dropsondes(dropsonde_path):
    
# Ludo's version
#     dropsondes = xr.open_dataset(dropsonde_path)

#     dropsondes = dropsondes.drop_vars(["trajectory","w_wind", "dz", "vt", "alt", "reference_time", \
#                                    "reference_pres", "reference_tdry", "reference_rh", "reference_wspd",\
#                                  "reference_wdir", "reference_lat", "reference_lon", "reference_alt", "T_interp",\
#                                  "T", "rh_interp", "q"])

#     dropsondes["time"] = dropsondes["time"].astype("datetime64")
#     dropsondes.rename({"Platform":"platform"})

# Ben's version

    dropsondes = xr.open_dataset(dropsonde_path)

    dropsondes = dropsondes.drop_vars(["trajectory","w_wind", "dz", "vt", "reference_time", \
                                       "reference_pres", "reference_tdry", "reference_rh", "reference_wspd",\
                                     "reference_wdir", "reference_lat", "reference_lon", "reference_alt", "T_interp",\
                                     "T", "rh_interp", "q"])

    dropsondes = dropsondes.drop_vars('gpsalt')
    dropsondes = dropsondes.rename_dims({'alt':'gpsalt'})
    dropsondes.rename({'alt':'gpsalt'})

# End Ben's version
    
    dropsondes["tdry"].attrs["units"] = "degC"
    dropsondes["dp"].attrs["units"] = "degC"
    dropsondes["pres"].attrs["units"] = "hPa"

    dropsondes["ascent_flag"] = (["launch_time"], np.zeros(len(dropsondes.launch_time.values)))

    return dropsondes

In [25]:
def prepare_radiosondes(radiosonde_path):

    radiosondes = xr.open_dataset(radiosonde_path)

    radiosondes = radiosondes.reset_coords(['lon','lat','time'])   

    radiosondes["tdry"].attrs["units"] = "degC"
    radiosondes["pres"].attrs["units"] = "hPa"
    radiosondes["dp"].attrs["units"] = "degC"
    
    theta = mpcalc.potential_temperature(radiosondes["pres"], radiosondes["tdry"])
    radiosondes["theta"] = (["launch_time", "gpsalt"], theta.magnitude)

    theta_v = mpcalc.virtual_potential_temperature(radiosondes["pres"], radiosondes["tdry"], radiosondes["mr"]/1000)
    radiosondes["theta_v"] = (["launch_time", "gpsalt"], theta_v.magnitude)

    theta_e = mpcalc.equivalent_potential_temperature(radiosondes["pres"], radiosondes["tdry"], radiosondes["dp"])
    radiosondes["theta_e"] = (["launch_time", "gpsalt"], theta_e.magnitude)

    del radiosondes.attrs["title"]
    del radiosondes.attrs["doi"]
    del radiosondes.attrs["surface_altitude"]
    del radiosondes.attrs["featureType"]
    del radiosondes.attrs["Conventions"]
    del radiosondes.attrs["history"]
    del radiosondes.attrs["NCO"]
    del radiosondes.attrs["nco_openmp_thread_number"]

    radiosondes = radiosondes.drop_vars(["specific_humidity", "ascent_rate"])

    length = len(radiosondes.launch_time.values)
    list_str = []
    for i in range(length):
        radiosonde = radiosondes.isel(launch_time = i)

        if (radiosonde.platform.values==1):
            list_str.append("BCO")
        elif (radiosonde.platform.values==2):
            list_str.append("MET")
        elif (radiosonde.platform.values==3):
            list_str.append("RHB")
        elif (radiosonde.platform.values==4):
            list_str.append("MER")
        else:
            list_str.append("ATL")     

    array = np.array(list_str, dtype=object)
    radiosondes["platform"] = (["launch_time"], array)
        
    return radiosondes

In [26]:
xr.open_dataset(dropsonde_path)

<xarray.Dataset>
Dimensions:         (alt: 1001, launch_time: 1140)
Coordinates:
  * launch_time     (launch_time) datetime64[ns] 2020-01-15T15:19:15 ... 2020-02-18T16:18:02
  * alt             (alt) int64 0 10 20 30 40 50 ... 9960 9970 9980 9990 10000
Data variables:
    trajectory      (launch_time) |S8 ...
    pres            (launch_time, alt) float64 ...
    tdry            (launch_time, alt) float64 ...
    dp              (launch_time, alt) float64 ...
    rh              (launch_time, alt) float64 ...
    u_wind          (launch_time, alt) float64 ...
    v_wind          (launch_time, alt) float64 ...
    w_wind          (launch_time, alt) float64 ...
    wspd            (launch_time, alt) float64 ...
    wdir            (launch_time, alt) float64 ...
    dz              (launch_time, alt) float64 ...
    mr              (launch_time, alt) float64 ...
    vt              (launch_time, alt) float64 ...
    theta           (launch_time, alt) float64 ...
    theta_e         (launch_time, alt) float64 ...
    theta_v         (launch_time, alt) float64 ...
    lat             (launch_time, alt) float64 ...
    lon             (launch_time, alt) float64 ...
    gpsalt          (launch_time, alt) float64 ...
    reference_time  (launch_time) datetime64[ns] ...
    reference_pres  (launch_time) float32 ...
    reference_tdry  (launch_time) float32 ...
    reference_rh    (launch_time) float32 ...
    reference_wspd  (launch_time) float32 ...
    reference_wdir  (launch_time) float32 ...
    reference_lat   (launch_time) float32 ...
    reference_lon   (launch_time) float32 ...
    reference_alt   (launch_time) float32 ...
    T_interp        (launch_time, alt) float64 ...
    T               (launch_time, alt) float64 ...
    rh_interp       (launch_time, alt) float64 ...
    q               (launch_time, alt) float64 ...
    iwv             (launch_time) float64 ...
    platform        (launch_time) object ...
    wind_dir        (launch_time, alt) float64 ...

In [27]:
dropsondes = prepare_dropsondes(dropsonde_path)
radiosondes = prepare_radiosondes(radiosonde_path)
print(dropsondes, radiosondes)

<xarray.Dataset>
Dimensions:      (gpsalt: 1001, launch_time: 1140)
Coordinates:
  * launch_time  (launch_time) datetime64[ns] 2020-01-15T15:19:15 ... 2020-02-18T16:18:02
    alt          (gpsalt) int64 0 10 20 30 40 50 ... 9960 9970 9980 9990 10000
Dimensions without coordinates: gpsalt
Data variables:
    pres         (launch_time, gpsalt) float64 ...
    tdry         (launch_time, gpsalt) float64 ...
    dp           (launch_time, gpsalt) float64 ...
    rh           (launch_time, gpsalt) float64 ...
    u_wind       (launch_time, gpsalt) float64 ...
    v_wind       (launch_time, gpsalt) float64 ...
    wspd         (launch_time, gpsalt) float64 ...
    wdir         (launch_time, gpsalt) float64 ...
    mr           (launch_time, gpsalt) float64 ...
    theta        (launch_time, gpsalt) float64 ...
    theta_e      (launch_time, gpsalt) float64 ...
    theta_v      (launch_time, gpsalt) float64 ...
    lat          (launch_time, gpsalt) float64 ...
    lon          (launch_time, g

In [28]:
# dropsondes = xr.open_dataset(dropsonde_path)

# dropsondes = dropsondes.drop_vars(["trajectory","w_wind", "dz", "vt", "reference_time", \
#                                    "reference_pres", "reference_tdry", "reference_rh", "reference_wspd",\
#                                  "reference_wdir", "reference_lat", "reference_lon", "reference_alt", "T_interp",\
#                                  "T", "rh_interp", "q"])

# dropsondes = dropsondes.drop_vars('gpsalt')
# dropsondes = dropsondes.rename_dims({'alt':'gpsalt'})
# dropsondes.rename({'alt':'gpsalt'})
# # dropsondes

In [29]:
all_sondes = xr.concat([dropsondes,radiosondes],dim='launch_time')

ValueError: arguments without labels along dimension 'gpsalt' cannot be aligned because they have different dimension size(s) {1001} than the size of the aligned dimension labels: 3000

In [19]:
print(all_sondes)

<xarray.Dataset>
Dimensions:      (gpsalt: 3000, launch_time: 2586)
Coordinates:
  * gpsalt       (gpsalt) float64 0.0 10.0 20.0 ... 2.998e+04 2.999e+04
  * launch_time  (launch_time) datetime64[ns] 2020-01-15T15:19:15 ... 2020-02-16T04:14:33
Data variables:
    time         (launch_time, gpsalt) datetime64[ns] NaT NaT NaT ... NaT NaT
    pres         (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    tdry         (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    dp           (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    rh           (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    u_wind       (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    v_wind       (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    wspd         (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    wdir         (launch_time, gpsalt) float64 nan nan nan nan ... nan nan nan
    mr           (launch_time, 

In [21]:
all_sondes.to_netcdf(os.path.join(inputdir, "all_sondes.nc"))

/Users/bfildier/anaconda3/envs/pyLMD/lib/python3.7/site-packages/ipykernel_launcher.py:1: SerializationWarning: saving variable gpsalt with floating point data as an integer dtype without any _FillValue to use for NaNs
  """Entry point for launching an IPython kernel.
